In [26]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import re #This allows us to search for patterns

In [34]:
def create_folders(folder_list):
    for f in folder_list:
        if not os.path.isdir(f):
            os.mkdir(f)

## For videos
If output is directory: Save result as images
If output is FILE: Save result as video

In [27]:
def modify_video(_input , output,kernel,th_low=150,th_up=255, name_prefix=''):    

    
    # Python program to save a  
    # video using OpenCV 
    
    
    import cv2 
    
    # Detect if the output path is a directory or a file
    is_directory = os.path.isdir(output)

    # Create an object to read  
    # from camera 
    video = cv2.VideoCapture(_input) 
    
    # We need to check if camera 
    # is opened previously or not 
    if (video.isOpened() == False):  
        print("Error reading video file") 
    
    # We need to set resolutions. 
    # so, convert them from float to integer. 
    frame_width = int(video.get(3)) 
    frame_height = int(video.get(4)) 
    
    size = (frame_width, frame_height) 
    
    # Below VideoWriter object will create 
    # a frame of above defined The output  
    # is stored in 'filename.avi' file.
    if(not is_directory):
        result = cv2.VideoWriter(output,  
                                cv2.VideoWriter_fourcc(*'MP4V'), 
                                30, size) 
    n_frame=0
    while(True): 
        ret, frame = video.read() 
    
        if ret == True: 
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _,thresh = cv2.threshold(frame,th_low,th_up,cv2.THRESH_BINARY)

            # opening = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)
            close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
            # Write the frame into the 
            # file 'filename.avi' 
            if(is_directory):
                
                #This part could be changed deppending on the naming conventions <----------------------
                
                
                out_img = '{}{}.png'.format(name_prefix,str(n_frame).zfill(3))
                #--------------------------------------------------------
                out_img = os.path.join(output, out_img)
                cv2.imwrite(out_img, close)
            else:
                result.write(close) 
    
            # Display the frame 
            # saved in the file 
            # cv2.imshow('Frame', close) 

            # Press S on keyboard  
            # to stop the process 
            if cv2.waitKey(1) & 0xFF == ord('s'): 
                break
            n_frame+=1
    
        # Break the loop 
        else: 
            break
    
    # When everything done, release  
    # the video capture and video  
    # write objects
    video.release()
    if (not is_directory): 
        result.release() 
        
    # Closes all the frames 
    cv2.destroyAllWindows()
    
    
    print("{} was successfully saved".format(output))

## Saving MANY videos

In [15]:
input_folder = '/home/josmar/proyectos/Background_Matting/walking_clips/krita_files'
kernel = np.ones((5,5),np.uint8)
th_low = 100
th_up = 255
out_folder = 'bin_close_images'
video_list = os.listdir(input_folder)
video_list.sort()

for v in video_list:
    output = '{}_close.mp4'.format(v.split('.')[0])
    output = os.path.join(out_folder,output)
    _input = os.path.join(input_folder,v)
    modify_video(_input, output, kernel, th_low, th_up)

10_720_correction.mp4
12_720_correction.mp4
13_720_correction.mp4
14_720_correction.mp4
15_720_correction.mp4
16_720_correction.mp4
18_720_correction.mp4
19_720_correction.mp4
20_720_correction.mp4
22_720_correction.mp4
23_720_correction.mp4
24_720_correction.mp4
25_720_correction.mp4
26_720_correction.mp4
27_720_correction.mp4
29_720_correction.mp4
2_720_correction.mp4
30_720_correction.mp4
31_720_correction.mp4
32_720_correction.mp4
34_720_correction.mp4
35_720_correction.mp4
36_720_correction.mp4
37_720_correction.mp4
39_720_correction.mp4
3_720_correction.mp4
40_720_correction.mp4
43_720_correction.mp4
44_720_correction.mp4
45_720_correction.mp4
46_720_correction.mp4
47_720_correction.mp4
4_720_correction.mp4
50_720_correction.mp4
52_720_correction.mp4
53_720_correction.mp4
5_720_correction.mp4
6_720_correction.mp4
7_720_correction.mp4
8_720_correction.mp4
9_720_correction.mp4


## Saving as folder
You need to create the folder

In [18]:
my_input = '/home/josmar/proyectos/Background_Matting/walking_clips/krita_files/27_720_correction.mp4'
kernel = np.ones((5,5),np.uint8)
th_low = 150
th_up = 255
output = 'bin_close_images/input'
if not os.path.isdir(output):
    os.mkdir(output)

modify_video(my_input, output, kernel, th_low, th_up)

bin_close_images/input was successfully saved


## Saving many folders

In [28]:
input_folder = '/home/josmar/proyectos/Background_Matting/walking_clips/krita_files'
kernel = np.ones((5,5),np.uint8)
th_low = 100
th_up = 255
out_folder = 'bin_close_images'
video_list = os.listdir(input_folder)
video_list.sort()

for v in video_list:
    n_subject = v.split('_')[0].zfill(3)
    subject_folder = os.path.join(out_folder,n_subject)
    output = os.path.join(subject_folder,'silhouette')
    
    #Create folder and subfolder
    for f in [subject_folder, output]:
        if not os.path.isdir(f):
            os.mkdir(f)
    
    _input = os.path.join(input_folder,v)
    modify_video(_input, output, kernel, th_low, th_up,n_subject+'-')

bin_close_images/010/silhouette was successfully saved
bin_close_images/012/silhouette was successfully saved
bin_close_images/013/silhouette was successfully saved
bin_close_images/014/silhouette was successfully saved
bin_close_images/015/silhouette was successfully saved
bin_close_images/016/silhouette was successfully saved
bin_close_images/018/silhouette was successfully saved
bin_close_images/019/silhouette was successfully saved
bin_close_images/020/silhouette was successfully saved
bin_close_images/022/silhouette was successfully saved
bin_close_images/023/silhouette was successfully saved
bin_close_images/024/silhouette was successfully saved
bin_close_images/025/silhouette was successfully saved
bin_close_images/026/silhouette was successfully saved
bin_close_images/027/silhouette was successfully saved
bin_close_images/029/silhouette was successfully saved
bin_close_images/002/silhouette was successfully saved
bin_close_images/030/silhouette was successfully saved
bin_close_

## Create images folder (This is extra)

In [42]:
import cv2 
def extract_frames(video_path, out_folder, name_prefix):
    # Create an object to read  
    # from video 
    video = cv2.VideoCapture(video_path) 
    
    # We need to check if camera 
    # is opened previously or not 
    if (video.isOpened() == False):  
        print("Error reading video file") 
    
    # We need to set resolutions. 
    # so, convert them from float to integer. 
    frame_width = int(video.get(3)) 
    frame_height = int(video.get(4)) 
    
    size = (frame_width, frame_height) 
    
    # Below VideoWriter object will create 
    # a frame of above defined The output  
    # is stored in 'filename.avi' file. 
    n_frame = 0
    while(True): 
        ret, frame = video.read() 
    
        if ret == True:   
    
            # Display the frame 
            # saved in the file 
            frame_str = str(n_frame).zfill(3)
            filename = '{}{}.jpg'.format(name_prefix,str(n_frame).zfill(3))
            filename = os.path.join(out_folder,filename)
            cv2.imwrite(filename, frame)  
    
            # Press S on keyboard  
            # to stop the process 
            if cv2.waitKey(1) & 0xFF == ord('s'): 
                break
            n_frame+=1
        # Break the loop 
        else: 
            break
    
    # When everything done, release  
    # the video capture and video  
    # write objects 
    video.release()  
        
    
    print("{} was successfully saved".format(video_path)) 

In [46]:
root_folder = '/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images'
video_list = os.listdir(root_folder)
video_list.sort()
for v in video_list:

    video_path = '/home/josmar/proyectos/Background_Matting/walking_clips/results/{}/1280x720/trimmed.mp4'.format(int(v))
    out_folder = '{}/{}/img'.format(root_folder,v)
    name_prefix = '{}-'.format(v)
    # Create folder and subfolder
    create_folders([out_folder])
    extract_frames(video_path, out_folder, name_prefix)

/home/josmar/proyectos/Background_Matting/walking_clips/results/2/1280x720/trimmed.mp4 was successfully saved
/home/josmar/proyectos/Background_Matting/walking_clips/results/3/1280x720/trimmed.mp4 was successfully saved
/home/josmar/proyectos/Background_Matting/walking_clips/results/4/1280x720/trimmed.mp4 was successfully saved
/home/josmar/proyectos/Background_Matting/walking_clips/results/5/1280x720/trimmed.mp4 was successfully saved
/home/josmar/proyectos/Background_Matting/walking_clips/results/6/1280x720/trimmed.mp4 was successfully saved
/home/josmar/proyectos/Background_Matting/walking_clips/results/7/1280x720/trimmed.mp4 was successfully saved
/home/josmar/proyectos/Background_Matting/walking_clips/results/8/1280x720/trimmed.mp4 was successfully saved
/home/josmar/proyectos/Background_Matting/walking_clips/results/9/1280x720/trimmed.mp4 was successfully saved
/home/josmar/proyectos/Background_Matting/walking_clips/results/10/1280x720/trimmed.mp4 was successfully saved
/home/jos

In [54]:
root_folder = '/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images'
video_list = os.listdir(root_folder)
video_list.sort()
for v in video_list:
    _file = '/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/{}/silhouette/{}-300.png'.format(v,v)
    
    if os.path.exists(_file):
        os.remove(_file)
        print(_file, " removed")
    else:
        print(_file, " doesn't exist, not removed")

/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/002/silhouette/002-300.png  doesn't exist, not removed
/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/003/silhouette/003-300.png  removed
/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/004/silhouette/004-300.png  removed
/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/005/silhouette/005-300.png  removed
/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/006/silhouette/006-300.png  removed
/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/007/silhouette/007-300.png  removed
/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/008/silhouette/008-300.png  removed
/home/josmar/proyectos/codes/annotation_tools/background_substraction/bin_close_images/009/silhouette/009-300.png  remov